In [1]:
# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
import math
%matplotlib inline 

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('VF_Data2.csv') 
print(df.shape)
df.describe().transpose()

(1200, 21)


,count,mean,std,min,25%,50%,75%,max
Season,1200.0,2.037866e+06,938.985508,2035992.0,2038248.00,2038304.0,2038362.0,2038422.0
Matchday,1200.0,1.550000e+01,8.659050,1.0,8.00,15.5,23.0,30.0
HomePos,1200.0,8.313333e+00,4.788451,0.0,4.00,8.0,13.0,16.0
AwayPos,1200.0,8.126667e+00,4.772624,0.0,4.00,8.0,12.0,16.0
HW,1200.0,5.398333e+00,4.213208,0.0,2.00,5.0,8.0,20.0
HD,1200.0,3.660833e+00,2.719123,0.0,2.00,3.0,6.0,14.0
HL,1200.0,5.457500e+00,4.178845,0.0,2.00,5.0,8.0,19.0
HGF,1200.0,1.915000e+01,13.729394,0.0,8.00,17.0,28.0,62.0
HGA,1200.0,1.920667e+01,12.647025,0.0,9.00,18.0,28.0,60.0
HGD,1200.0,-5.666667e-02,10.708440,-34.0,-6.25,0.0,6.0,28.0


In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [4]:
df['h_team']=le.fit_transform(df.HomeTeam) 
df['a_team']=le.fit_transform(df.AwayTeam)
df['ftr']=le.fit_transform(df.FTR)
df.head(16)

,Season,Matchday,HomeTeam,AwayTeam,HomePos,AwayPos,HW,HD,HL,HGF,...,AD,AL,AGF,AGA,AGD,APoints,FTR,h_team,a_team,ftr
0,2035992,1,che,mbl,0,0,0,0,0,0,...,0,0,0,0,0,0,h,2,8,2
1,2035992,1,west,bur,0,0,0,0,0,0,...,0,0,0,0,0,0,a,14,1,0
2,2035992,1,tot,mre,0,0,0,0,0,0,...,0,0,0,0,0,0,h,12,9,2
3,2035992,1,eve,liv,0,0,0,0,0,0,...,0,0,0,0,0,0,h,4,6,2
4,2035992,1,new,kiev,0,0,0,0,0,0,...,0,0,0,0,0,0,a,10,5,0
5,2035992,1,lond,bou,0,0,0,0,0,0,...,0,0,0,0,0,0,h,7,0,2
6,2035992,1,wat,sou,0,0,0,0,0,0,...,0,0,0,0,0,0,h,13,11,2
7,2035992,1,cry,wol,0,0,0,0,0,0,...,0,0,0,0,0,0,a,3,15,0
8,2035992,2,liv,wat,14,8,0,0,1,0,...,0,0,2,1,1,3,a,6,13,0
9,2035992,2,bou,new,11,13,0,0,1,1,...,0,1,0,1,-1,0,h,0,10,2


In [5]:
df.drop(['Season','HomeTeam','AwayTeam','FTR'],axis=1,inplace=True)
df.head(5)

,Matchday,HomePos,AwayPos,HW,HD,HL,HGF,HGA,HGD,HPoints,AW,AD,AL,AGF,AGA,AGD,APoints,h_team,a_team,ftr
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,2
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,1,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,9,2
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,6,2
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,5,0


In [9]:
target_column = ['ftr'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Matchday,1200.0,0.516667,0.288635,0.033333,0.266667,0.516667,0.766667,1.0
HomePos,1200.0,0.519583,0.299278,0.000000,0.250000,0.500000,0.812500,1.0
AwayPos,1200.0,0.507917,0.298289,0.000000,0.250000,0.500000,0.750000,1.0
HW,1200.0,0.269917,0.210660,0.000000,0.100000,0.250000,0.400000,1.0
HD,1200.0,0.261488,0.194223,0.000000,0.142857,0.214286,0.428571,1.0
HL,1200.0,0.287237,0.219939,0.000000,0.105263,0.263158,0.421053,1.0
HGF,1200.0,0.308871,0.221442,0.000000,0.129032,0.274194,0.451613,1.0
HGA,1200.0,0.320111,0.210784,0.000000,0.150000,0.300000,0.466667,1.0
HGD,1200.0,-0.002024,0.382444,-1.214286,-0.223214,0.000000,0.214286,1.0
HPoints,1200.0,0.310247,0.221857,0.000000,0.125000,0.281250,0.468750,1.0


In [20]:
X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=40)
print(X_train.shape); print(X_test.shape)

(900, 19)
(300, 19)


In [32]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=1200)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)


from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

from collections import OrderedDict
new_data=OrderedDict([
    ('Matchday',    ),
    ('h_team',    ),
    ('HomePos',   ),
    ('HW',    ),
    ('HD',    ),
    ('HL',    ),
    ('HGF',    ),
    ('HGA',    ),
    ('HGD',      ),
    ('HPoints',    ),
    ('a_team',     ),
    ('AwayPos',    ),
    ('AW',      ),
    ('AD',    ),
    ('AL',    ),
    ('AGF',    ),
    ('AGA',    ),
    ('AGD',      ),
    ('APoints',    )
])
# .values.reshape(1, -1) bcoz it must be 2--dim, bcoz we passed only one new observation
new_data=pd.Series(new_data).values.reshape(1,-1)

predict_test = mlp.predict(new_data)
predict_test

C:\Users\Charleagle\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[[134  18 121]
 [ 57  34 154]
 [ 48  36 298]]
              precision    recall  f1-score   support

           0       0.56      0.49      0.52       273
           1       0.39      0.14      0.20       245
           2       0.52      0.78      0.62       382

    accuracy                           0.52       900
   macro avg       0.49      0.47      0.45       900
weighted avg       0.50      0.52      0.48       900



array([2])

In [19]:
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

ValueError: Found input variables with inconsistent numbers of samples: [300, 1]

In [ ]:
new_data=OrderedDict([
    ('Matchday',    ),
    ('h_team',    ),
    ('HomePos',   ),
    ('HW',    ),
    ('HD',    ),
    ('HL',    ),
    ('HGF',    ),
    ('HGA',    ),
    ('HGD',      ),
    ('HPoints',    ),
    ('a_team',     ),
    ('AwayPos',    ),
    ('AW',      ),
    ('AD',    ),
    ('AL',    ),
    ('AGF',    ),
    ('AGA',    ),
    ('AGD',      ),
    ('APoints',    )
])